In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_data = pd.read_csv('cure_the_princess_train.csv')
valid_data = pd.read_csv('cure_the_princess_validation.csv')
test_data = pd.read_csv('cure_the_princess_test.csv')

classes = 14

train_X = torch.tensor(train_data.iloc[:, :-1].values, dtype=torch.float32)
train_y = torch.tensor(train_data.iloc[:, -1].values, dtype=torch.long)

valid_X = torch.tensor(valid_data.iloc[:, :-1].values, dtype=torch.float32)
valid_y = torch.tensor(valid_data.iloc[:, -1].values, dtype=torch.long)

test_X = torch.tensor(valid_data.iloc[:, :-1].values, dtype=torch.float32)
test_y = torch.tensor(valid_data.iloc[:, -1].values, dtype=torch.long)

class MLP(nn.Module):
    def __init__(self, num_input_feats, num_hidden_neuron1,a, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(train_X.shape[1], 100)  # ilk hidden layer: 100 noron
        self.relu1 = nn.ReLU()          # ilk hidden layer aktivasyon fonk: ReLU
        self.fc2 = nn.Linear(100, 50)   # ikinci hidden layer: 50 noron
        self.relu2 = nn.ReLU()          # ikinci hidden layer aktivasyon fonk: ReLU
        self.fc3 = nn.Linear(50, 1)     # output layer: 1 noron
        self.sigmoid = nn.Sigmoid()     # output layer aktivasyon fonk: sigmoid

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

learning_rate = 0.01
# Hata fonksiyonu olarak binary cross entropy loss kullanacağız.
criterion = nn.BCELoss()

# Optimizasyon algoritması olarak stochastic gradient descent kullanacağız.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size=16
# Batch size 16 olduğu için, DataLoader'a 16 veri noktası eklememiz gerekiyor.
train_dataset = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = TensorDataset(valid_X, valid_y)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 10
SEED = 190401006
torch.manual_seed(SEED)

train_losses = []
valid_losses = []
train_accs = []
valid_accs = []

for epoch in range(num_epochs):
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_acc += ((outputs > 0.5).int() == targets.int().view(-1, 1)).sum().item()
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validate
    model.eval()
    valid_loss = 0.0
    valid_acc = 0.0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(valid_loader):
            outputs = model(inputs)
            loss = criterion(outputs, targets.float().view(-1, 1))
            valid_loss += loss.item() * inputs.size(0)
            valid_acc += ((outputs > 0.5).int() == targets.int().view(-1, 1)).sum().item()
        valid_loss /= len(valid_loader.dataset)
        valid_acc /= len(valid_loader.dataset)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')

# Test
model.eval()
test_loss = 0.0
test_acc = 0.0
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
        outputs = model(inputs)
        loss = criterion(outputs, targets.float().view(-1, 1))
        test_loss += loss.item() * inputs.size(0)
        test_acc += ((outputs > 0.5).int() == targets.int().view(-1, 1)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_acc /= len(test_loader.dataset)

print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

Epoch 1/10, Train Loss: 0.5836, Train Acc: 0.6957, Valid Loss: 0.4091, Valid Acc: 0.8312
Epoch 2/10, Train Loss: 0.4196, Train Acc: 0.8315, Valid Loss: 0.5739, Valid Acc: 0.6433
Epoch 3/10, Train Loss: 0.3693, Train Acc: 0.8339, Valid Loss: 0.2881, Valid Acc: 0.8822
Epoch 4/10, Train Loss: 0.2978, Train Acc: 0.8906, Valid Loss: 0.2108, Valid Acc: 0.9236
Epoch 5/10, Train Loss: 0.2822, Train Acc: 0.8946, Valid Loss: 0.2680, Valid Acc: 0.8726
Epoch 6/10, Train Loss: 0.2500, Train Acc: 0.9050, Valid Loss: 0.2190, Valid Acc: 0.9045
Epoch 7/10, Train Loss: 0.2496, Train Acc: 0.9010, Valid Loss: 0.2563, Valid Acc: 0.8854
Epoch 8/10, Train Loss: 0.2275, Train Acc: 0.9073, Valid Loss: 0.3763, Valid Acc: 0.8185
Epoch 9/10, Train Loss: 0.2075, Train Acc: 0.9265, Valid Loss: 0.1583, Valid Acc: 0.9490
Epoch 10/10, Train Loss: 0.1954, Train Acc: 0.9257, Valid Loss: 0.1654, Valid Acc: 0.9490
Test Loss: 0.1654, Test Acc: 0.9490


In [46]:
images.shape

torch.Size([16, 13])

In [42]:
images

tensor([[23.6000, 10.3000,  4.6000,  5.5000,  2.8000, 14.4000,  9.1000, 10.5000,
         21.6000, 12.1000,  4.3000,  3.0000, 29.3000],
        [ 4.0000, 12.1000, 17.8000, 25.2000, 16.9000,  4.2000,  7.2000, 17.8000,
         15.4000, 32.1000, 23.1000,  3.1000, 10.2000],
        [24.9000, 15.6000, 14.2000, 17.7000, 19.7000,  1.2000, 18.2000, 26.6000,
          6.0000, 14.9000,  9.9000, 10.5000, 16.5000],
        [22.7000,  8.9000, 27.9000, 21.2000,  9.5000,  7.4000,  7.2000, 20.1000,
         30.1000, 30.1000,  4.0000,  6.4000, 10.4000],
        [14.4000, 31.0000, 20.6000,  4.2000, 16.2000, 16.7000,  6.9000, 20.1000,
         22.7000, 19.9000, 13.2000,  4.9000,  4.8000],
        [ 8.7000, 29.6000, 17.0000, 17.7000, 24.5000, 29.3000, 11.5000, 33.2000,
         28.5000,  3.3000, 10.8000, 16.5000,  6.3000],
        [ 5.4000, 16.6000,  3.4000, 10.6000, 13.1000, 13.4000, 12.6000, 13.8000,
          6.2000, 18.8000,  7.6000, 12.5000,  8.2000],
        [29.1000, 15.8000, 18.7000, 14.5000, 31.

In [37]:
images.size()

torch.Size([16, 13])